In [3497]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy
from scipy.fftpack import fft, fftfreq, fftshift, ifft
from scipy import signal
from scipy.signal import butter, lfilter, filtfilt
from optcutfreq import optcutfreq
import math 
from IPython.display import display, Math, Latex
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.signal import find_peaks
from detecta import detect_peaks
import math
from detecta import detect_onset
from tnorma import tnorma
from scipy.interpolate import CubicSpline
from scipy.signal import argrelextrema
import random
from scipy.integrate import cumtrapz
from scipy import integrate
from scipy.signal import detrend
from scipy.interpolate import interp1d, PchipInterpolator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr



**Preprocessing**

In [3498]:
def getNA(df):
    '''
        return index of rows with na in df
    '''
    rows_with_na = df[df.isna().any(axis=1)].index

    return list(rows_with_na)

In [3499]:
#no exo
no_exo = pd.read_csv('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/Results.csv')
no_exo = no_exo.drop(['Unnamed: 0'], axis = 1)
no_exo = no_exo.drop([6], axis = 0)
no_exo = no_exo.reset_index(drop=True)
    # Save info
info = no_exo[['ID',	'Age',	'Height (m)',	'Weight (kg)',	'BMI (kg/m2)']]
#SRC
SRC = pd.read_csv('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/Results2.csv')
SRC = SRC.drop(['Unnamed: 0'], axis = 1)
SRC = pd.concat([info, SRC], axis=1)
# HPT
HPT = pd.read_csv('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/Results3.csv')
HPT = HPT.drop(['Unnamed: 0'], axis = 1)
HPT = pd.concat([info, HPT], axis=1)
# MR
MR = pd.read_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Energetics_Full_Data.xlsx', usecols=['Subjects', 'Condition', 'Net MR [W/kg]'])
MR_no_exo = MR.loc[(MR['Condition']=='CTR_0%')].reset_index(drop = True)[['Net MR [W/kg]']]
MR_SRC = MR.loc[(MR['Condition']=='SRC_0%')].reset_index(drop = True)[['Net MR [W/kg]']]
MR_HPT = MR.loc[(MR['Condition']=='HPT_0%')].reset_index(drop = True)[['Net MR [W/kg]']]
# MR= MR.loc[(MR['Condition']=='CTR_0%')| (MR['Condition']=='SRC_0%')| (MR['Condition']=='HPT_0%')].reset_index(drop = True)

# Perceived Effort 
Borg = pd.read_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Borg.xlsx', usecols=['Subject', 'NO_flat', 'SRC_flat', 'HPT_flat'])

#Clean 
to_drop = list(set(getNA(no_exo)+ getNA(SRC) + getNA(HPT) + getNA(MR_no_exo) + getNA(MR_HPT) + getNA(MR_SRC) + getNA(Borg)))
no_exo, SRC, HPT, MR_no_exo, MR_SRC, MR_HPT, Borg = no_exo.drop(to_drop, axis = 0).reset_index(drop = True), SRC.drop(to_drop, axis = 0).reset_index(drop = True), HPT.drop(to_drop, axis = 0).reset_index(drop = True), MR_no_exo.drop(to_drop, axis = 0).reset_index(drop = True), MR_SRC.drop(to_drop, axis = 0).reset_index(drop = True), MR_HPT.drop(to_drop, axis = 0).reset_index(drop = True), Borg.drop(to_drop, axis = 0).reset_index(drop = True)
# Resp with best = (SRC)
Best = SRC


# GET RESP 


print(f'{len(SRC)} participants finally included')

MR_change = pd.DataFrame(MR_no_exo['Net MR [W/kg]'].values - MR_SRC['Net MR [W/kg]'].values, columns = ['red'])


Non_resp_id = np.where(MR_no_exo['Net MR [W/kg]'].values - MR_SRC['Net MR [W/kg]'].values <np.mean(MR_no_exo['Net MR [W/kg]'].values - MR_SRC['Net MR [W/kg]'].values))[0]
MR_Non_resp = MR_change.loc[list(Non_resp_id)]

resp_id = np.where(MR_no_exo['Net MR [W/kg]'].values - MR_SRC['Net MR [W/kg]'].values > np.mean(MR_no_exo['Net MR [W/kg]'].values - MR_SRC['Net MR [W/kg]'].values))[0]
MR_resp = MR_change.loc[list(resp_id)]

#DF to Save
Baseline_df = pd.concat([no_exo, MR_no_exo], axis=1) #baseline for PCA (1)
Perceived_df = pd.concat([SRC, Borg], axis=1) # Perceived for correlation (4)
SRC, HPT = SRC, HPT # SRC et HPT (2)
resp, non_resp = SRC.loc[list(resp_id)], SRC.loc[list(Non_resp_id)] #(3)
resp_noexo, non_resp_noexo = no_exo.loc[list(resp_id)], no_exo.loc[list(Non_resp_id)] #(3)
respborg, non_respborg = Borg.loc[list(resp_id)], Borg.loc[list(Non_resp_id)] #(3)
respMRSRC, non_respMRSRC = MR_SRC[['Net MR [W/kg]']].loc[list(resp_id)], MR_SRC[['Net MR [W/kg]']].loc[list(Non_resp_id)]

19 participants finally included


In [3500]:
resp

,ID,Age,Height (m),Weight (kg),BMI (kg/m2),mean_step_time (s)SRC,mean_stride_time (s)SRC,cadence (step/min)SRC,mean_swing_time (s)SRC,mean_stance_time (s)SRC,...,mean_GC_rms_acc_AP (mm/s2)SRC,cov_rms_acc_ML (%)SRC,cov_rms_acc_AP (%)SRC,linear_momentum_ML (kg.m/s)SRC,linear_momentum_AP (kg.m/s)SRC,est_Vert_vel_redir (rad)SRC,est_mean_mech_work_pushoff (J/kg)SRC,est_inst_power_pushoff (W)SRC,est_cov_mech_work_pushoff (%)SRC,est_cov_inst_power_pushoff (%)SRC
0,1,21,1.77,77.0,24.577867,0.606452,1.212816,99,0.434751,0.777895,...,817.306375,12.630689,3.846838,6.639640,5.080788,0.021346,44.276301,40.174009,15.175357,5.020653
1,2,22,1.75,68.0,22.204082,0.626762,1.253400,96,0.460493,0.792868,...,940.300428,19.615811,3.125972,5.252284,5.397460,0.026652,7.744360,40.154242,52.059146,5.116444
2,3,21,1.92,70.0,18.988715,0.576736,1.153444,104,0.406364,0.747073,...,741.582247,11.489341,4.410015,4.379101,4.095917,0.017406,44.286836,33.790119,9.729668,4.925594
3,4,24,1.72,63.0,21.295295,0.601915,1.203909,100,0.426426,0.777482,...,840.866919,9.036264,3.995260,3.552584,4.173861,0.010230,35.787143,34.995076,5.849535,6.170333
7,9,28,1.89,83.0,23.235632,0.617750,1.235475,97,0.455342,0.780201,...,785.159667,10.444686,3.734075,7.815274,5.329535,0.011728,36.537732,44.453324,13.509317,5.710927
9,11,25,1.81,77.0,23.503556,0.631359,1.262684,95,0.464452,0.798258,...,913.712823,9.022721,4.201492,6.984649,5.938653,0.011789,9.755115,47.523617,41.401303,6.499914
10,12,59,1.77,77.0,24.577867,0.591262,1.182580,101,0.434179,0.748381,...,818.065812,6.397960,3.640271,7.217133,4.929675,0.025483,30.755179,40.662525,16.245449,5.281022
12,14,27,1.77,71.5,22.822305,0.591304,1.182625,102,0.416322,0.766229,...,846.170066,10.785512,3.293254,3.835804,4.947529,0.009385,28.507236,41.038506,12.846504,4.090295
14,16,28,1.67,59.5,21.334576,0.575034,1.150038,104,0.411450,0.738669,...,840.871346,16.868033,3.038902,4.043805,3.795121,0.014102,11.928939,34.348006,50.594719,3.580044
17,21,44,1.80,72.5,22.376543,0.587498,1.174870,102,0.426628,0.748373,...,795.116473,16.431177,4.364449,4.163874,4.637897,0.022401,14.651311,39.159258,35.096746,7.926293


In [3501]:
resp_noexo

,ID,Age,Height (m),Weight (kg),BMI (kg/m2),mean_step_time (s)No_Exo,mean_stride_time (s)No_Exo,cadence (step/min)No_Exo,mean_swing_time (s)No_Exo,mean_stance_time (s)No_Exo,...,mean_GC_rms_acc_AP (mm/s2)No_Exo,cov_rms_acc_ML (%)No_Exo,cov_rms_acc_AP (%)No_Exo,linear_momentum_ML (kg.m/s)No_Exo,linear_momentum_AP (kg.m/s)No_Exo,est_Vert_vel_redir (rad)No_Exo,est_mean_mech_work_pushoff (J/kg)No_Exo,est_inst_power_pushoff (W)No_Exo,est_cov_mech_work_pushoff (%)No_Exo,est_cov_inst_power_pushoff (%)No_Exo
0,1,21,1.77,77.0,24.577867,0.594273,1.188560,101,0.422187,0.766397,...,785.875405,12.048174,3.920663,5.495082,4.903971,0.017013,40.762103,35.349868,13.098379,6.468719
1,2,22,1.75,68.0,22.204082,0.602688,1.205413,100,0.437788,0.767631,...,886.901629,16.436217,3.072319,4.323935,4.854290,0.019661,8.389796,38.634943,59.362560,3.230572
2,3,21,1.92,70.0,18.988715,0.552267,1.104468,108,0.392968,0.711454,...,673.617327,8.875483,3.657682,3.833874,3.480319,0.015618,33.013715,29.342462,13.221398,6.020458
3,4,24,1.72,63.0,21.295295,0.595097,1.190226,101,0.432266,0.757926,...,805.447544,8.131469,2.885782,2.283199,3.901399,0.010778,53.313141,31.866769,5.846666,4.686949
7,9,28,1.89,83.0,23.235632,0.592956,1.185854,101,0.432291,0.753549,...,712.766311,9.644278,3.856213,6.709390,4.595492,0.007705,28.662663,38.492205,10.206967,4.037836
9,11,25,1.81,77.0,23.503556,0.619499,1.239013,97,0.449189,0.789833,...,844.042368,11.085533,3.556351,4.592785,5.386542,0.008624,36.900547,41.093667,9.288757,5.780604
10,12,59,1.77,77.0,24.577867,0.575671,1.151496,104,0.413967,0.737431,...,782.276895,6.936429,3.532778,5.087984,4.533729,0.017592,5.409660,36.995841,20.509841,3.541281
12,14,27,1.77,71.5,22.822305,0.577408,1.154805,104,0.405426,0.749365,...,810.438067,13.502470,3.268759,3.043993,4.517127,0.010047,20.731821,36.436846,6.430961,3.414281
14,16,28,1.67,59.5,21.334576,0.569618,1.139322,105,0.400453,0.738818,...,845.799054,16.386040,4.398292,3.181925,3.724518,0.016680,20.400573,31.856283,28.125803,3.363000
17,21,44,1.80,72.5,22.376543,0.585377,1.170973,103,0.421034,0.749925,...,754.432165,15.900305,3.679350,3.044100,4.458297,0.018900,10.534173,34.191635,71.688235,5.242549


In [3502]:
SRC.columns

Index(['ID', 'Age', 'Height (m)', 'Weight (kg)', 'BMI (kg/m2)',
       'mean_step_time (s)SRC', 'mean_stride_time (s)SRC',
       'cadence (step/min)SRC', 'mean_swing_time (s)SRC',
       'mean_stance_time (s)SRC', 'mean_ds_time (s)SRC', 'mean_ss_time (s)SRC',
       'cov_step_time (%)SRC', 'cov_stride_time (%)SRC',
       'cov_swing_time (%)SRC', 'cov_stance_time (%)SRC',
       'swing_time_perctg (%)SRC', 'stance_time_perctg (%)SRC',
       'ds_time_perctg (%)SRC', 'ss_time_perctg (%)SRC',
       'mean_step_width (mm)SRC', 'mean_step_length (mm)SRC',
       'mean_stride_length (mm)SRC', 'cov_step_width (%)SRC',
       'cov_step_length (%)SRC', 'cov_stride_length (%)SRC',
       'mean_GC_range_ML (mm)SRC', 'mean_GC_range_AP (mm)SRC',
       'cov_range_ML (%)SRC', 'cov_range_AP (%)SRC', 'mean_vel_ML (mm/s)SRC',
       'mean_vel_AP (mm/s)SRC', 'mean_GC_rms_acc_ML (mm/s2)SRC',
       'mean_GC_rms_acc_AP (mm/s2)SRC', 'cov_rms_acc_ML (%)SRC',
       'cov_rms_acc_AP (%)SRC', 'linear_momentu

In [3503]:
# import pandas as pd

data = {
    'Parameter Name': [
        'mean_step_time (s)', 'mean_stride_time (s)', 'step frequency (step/min)',
        'mean_swing_time (s)', 'mean_stance_time (s)', 'mean_ds_time (s)',
        'mean_ss_time (s)', 'cov_step_time (%)', 'cov_stride_time (%)',
        'cov_swing_time (%)', 'cov_stance_time (%)', 'swing_time_perctg (%)',
        'stance_time_perctg (%)', 'ds_time_perctg (%)', 'ss_time_perctg (%)',
        'mean_step_width (mm)', 'mean_step_length (mm)',
        'mean_stride_length (mm)', 'cov_step_width (%)', 'cov_step_length (%)',
        'cov_stride_length (%)', 'mean_GC_COM_range_ML (mm)',
        'mean_GC_COM_range_AP (mm)', 'cov_range_ML_COM (%)', 'cov_range_AP_COM (%)',
        'mean_vel_COM_ML (mm/s)', 'mean_vel_COM_AP (mm/s)',
        'mean_GC_rms_acc_ML_COM (mm/s2)', 'mean_GC_rms_acc_AP_COM (mm/s2)',
        'cov_rms_acc_ML_COM (%)', 'cov_rms_acc_AP_COM (%)',
        'linear_momentum_ML_COM (kg.m/s)', 'linear_momentum_AP_COM (kg.m/s)',
        'est_Vert_vel_redir_COM_angle (rad)', 'est_mean_mech_work_pushoff (J/kg)',
        'est_inst_power_pushoff (W)', 'est_cov_mech_work_pushoff (%)',
        'est_cov_inst_power_pushoff (%)'
    ],
    'Definition': [
        'The average time taken for one complete step, from initial contact of one foot to the next initial contact of the opposite foot.',
        'The average time taken for one complete stride, from initial contact of one foot to the next initial contact of the same foot.',
        'The number of steps taken per minute.',
        'The average time during which the foot is off the ground (swing phase) during one gait cycle.',
        'The average time during which the foot is in contact with the ground (stance phase) during one gait cycle.',
        'The average time taken for the double stance phase, when both feet are in contact with the ground during one gait cycle.',
        'The average time taken for the single stance phase, when only one foot is in contact with the ground during one gait cycle.',
        'The coefficient of variation of step time, representing the variability in the duration of steps.',
        'The coefficient of variation of stride time, representing the variability in the duration of strides.',
        'The coefficient of variation of swing time, representing the variability in the duration of the swing phase.',
        'The coefficient of variation of stance time, representing the variability in the duration of the stance phase.',
        'The percentage of the gait cycle spent in the swing phase.',
        'The percentage of the gait cycle spent in the stance phase.',
        'The percentage of the gait cycle spent in the double stance phase.',
        'The percentage of the gait cycle spent in the single stance phase.',
        'The average mediolateral distance between the foot positions at initial contact during one gait cycle.',
        'The average anteroposterior distance covered by a step, from initial contact to the next initial contact of the same foot.',
        'The average anteroposterior distance covered by a stride, from initial contact of one foot to the next initial contact of the opposite foot.',
        'The coefficient of variation of step width, representing the variability in the mediolateral distance between the foot positions at initial contact.',
        'The coefficient of variation of step length, representing the variability in the anteroposterior distance covered by a step.',
        'The coefficient of variation of stride length, representing the variability in the anteroposterior distance covered by a stride.',
        'The average mediolateral range of displacement of the center of mass during one gait cycle.',
        'The average anteroposterior range of displacement of the center of mass during one gait cycle.',
        'The coefficient of variation of mediolateral displacement of the center of mass during the gait cycle.',
        'The coefficient of variation of anteroposterior displacement of the center of mass during the gait cycle.',
        'The average mediolateral velocity of the center of mass during one gait cycle.',
        'The average anteroposterior velocity of the center of mass during one gait cycle.',
        'The average root mean square acceleration in the mediolateral direction of the center of mass during one gait cycle.',
        'The average root mean square acceleration in the anteroposterior direction of the center of mass during one gait cycle.',
        'The coefficient of variation of the root mean square acceleration in the mediolateral direction of the center of mass during the gait cycle.',
        'The coefficient of variation of the root mean square acceleration in the anteroposterior direction of the center of mass during the gait cycle.',
        'The linear momentum in the mediolateral direction of the center of mass during one gait cycle.',
        'The linear momentum in the anteroposterior direction of the center of mass during one gait cycle.',
        'The estimated angle of redirection of the vertical velocity vector of the center of mass during one gait cycle.',
        'The estimated mean mechanical work per unit mass exerted during push-off phase in one gait cycle.',
        'The estimated instantaneous power exerted during push-off phase in one gait cycle.',
        'The coefficient of variation of the estimated mean mechanical work per unit mass exerted during push-off phase.',
        'The coefficient of variation of the estimated instantaneous power exerted during push-off phase.'
    ]
}

df = pd.DataFrame(data)
df.to_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/defparam.xlsx', index=False)


In [3504]:
import pandas as pd

# Define the parameter names and their corresponding full names
parameters = [
    ('mean_step_time', 'Mean Step Time'),
    ('mean_stride_time', 'Mean Stride Time'),
    ('step_frequency', 'Step Frequency'),
    ('mean_swing_time', 'Mean Swing Time'),
    ('mean_stance_time', 'Mean Stance Time'),
    ('mean_ds_time', 'Mean Double Stance Time'),
    ('mean_ss_time', 'Mean Single Stance Time'),
    ('cov_step_time', 'Coefficient of Variation of Step Time'),
    ('cov_stride_time', 'Coefficient of Variation of Stride Time'),
    ('cov_swing_time', 'Coefficient of Variation of Swing Time'),
    ('cov_stance_time', 'Coefficient of Variation of Stance Time'),
    ('swing_time_perctg', 'Swing Time Percentage'),
    ('stance_time_perctg', 'Stance Time Percentage'),
    ('ds_time_perctg', 'Double Stance Time Percentage'),
    ('ss_time_perctg', 'Single Stance Time Percentage'),
    ('mean_step_width', 'Mean Step Width'),
    ('mean_step_length', 'Mean Step Length'),
    ('mean_stride_length', 'Mean Stride Length'),
    ('cov_step_width', 'Coefficient of Variation of Step Width'),
    ('cov_step_length', 'Coefficient of Variation of Step Length'),
    ('cov_stride_length', 'Coefficient of Variation of Stride Length'),
    ('mean_GC_COM_range_ML', 'Mean Mediolateral Range of Displacement of the Center of Mass during Gait Cycle'),
    ('mean_GC_COM_range_AP', 'Mean Anteroposterior Range of Displacement of the Center of Mass during Gait Cycle'),
    ('cov_range_ML_COM', 'Coefficient of Variation of Mediolateral Displacement of the Center of Mass during Gait Cycle'),
    ('cov_range_AP_COM', 'Coefficient of Variation of Anteroposterior Displacement of the Center of Mass during Gait Cycle'),
    ('mean_vel_COM_ML', 'Mean Mediolateral Velocity of the Center of Mass during Gait Cycle'),
    ('mean_vel_COM_AP', 'Mean Anteroposterior Velocity of the Center of Mass during Gait Cycle'),
    ('mean_GC_rms_acc_ML_COM', 'Mean Root Mean Square Acceleration in Mediolateral Direction of the Center of Mass during Gait Cycle'),
    ('mean_GC_rms_acc_AP_COM', 'Mean Root Mean Square Acceleration in Anteroposterior Direction of the Center of Mass during Gait Cycle'),
    ('cov_rms_acc_ML_COM', 'Coefficient of Variation of Root Mean Square Acceleration in Mediolateral Direction of the Center of Mass during Gait Cycle'),
    ('cov_rms_acc_AP_COM', 'Coefficient of Variation of Root Mean Square Acceleration in Anteroposterior Direction of the Center of Mass during Gait Cycle'),
    ('linear_momentum_ML_COM', 'Mediolateral Linear Momentum of the Center of Mass during Gait Cycle'),
    ('linear_momentum_AP_COM', 'Anteroposterior Linear Momentum of the Center of Mass during Gait Cycle'),
    ('est_Vert_vel_redir_COM_angle', 'Estimated Angle of Redirection of Vertical Velocity Vector of the Center of Mass during Gait Cycle'),
    ('est_mean_mech_work_pushoff', 'Estimated Mean Mechanical Work per Unit Mass Exerted during Push-off Phase in Gait Cycle'),
    ('est_inst_power_pushoff', 'Estimated Instantaneous Power Exerted during Push-off Phase in Gait Cycle'),
    ('est_cov_mech_work_pushoff', 'Coefficient of Variation of Estimated Mean Mechanical Work per Unit Mass Exerted during Push-off Phase'),
    ('est_cov_inst_power_pushoff', 'Coefficient of Variation of Estimated Instantaneous Power Exerted during Push-off Phase')
]

# Create a DataFrame with the parameter names and full names
df = pd.DataFrame(parameters, columns=['Parameter', 'Full Name'])
df = df[['Full Name']]
# Display the DataFrame
df.to_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/Full_Name.xlsx', index=False)


**Stat Part 1**

In [3505]:
# MAn withney resp versus non resp
from scipy import stats
u_statistic, p_value = stats.mannwhitneyu(MR_resp.red, MR_Non_resp.red)
'significance ?' , p_value < 0.05

('significance ?', True)

In [3506]:
import numpy as np
from scipy.stats import f_oneway

# Example arrays
array1 = np.array([1, 2, 3, 4, 5])
array2 = np.array([2, 4, 6, 8, 10, 12, 14, 16])

# Perform F-test
_, p_value = f_oneway(MR_resp.red, MR_Non_resp.red)

# Set significance level
alpha = 0.05

# Compare p-value with significance level
if p_value > alpha:
    print("The variances are equal.")
else:
    print("The variances are not equal.")


The variances are not equal.


In [3507]:
p_value

3.72727137825942e-05

In [3508]:
# Paired T test SRC
    # Shapiro wilk 
import scipy.stats as stats

# Baseline measurements
baseline = MR_no_exo['Net MR [W/kg]'].values

# Follow-up measurements
follow_up = MR_SRC['Net MR [W/kg]'].values

# Perform Shapiro-Wilk test on baseline measurements
_, baseline_p_value = stats.shapiro(baseline)
print("Baseline Shapiro-Wilk p-value:", baseline_p_value)

# Perform Shapiro-Wilk test on follow-up measurements
_, follow_up_p_value = stats.shapiro(follow_up)
print("Follow-up Shapiro-Wilk p-value:", follow_up_p_value)
print('significance', follow_up_p_value>0.05, baseline_p_value>0.05)

    # paired t 
import scipy.stats as stats

# Perform paired t-test
t_statistic, p_value = stats.ttest_rel(baseline, follow_up)

# Print the results
print("Paired t-test:")
print("p-value:", p_value)
'significance', p_value<0.05

Baseline Shapiro-Wilk p-value: 0.9997049570083618
Follow-up Shapiro-Wilk p-value: 0.6025470495223999
significance True True
Paired t-test:
p-value: 0.004821636904519003


('significance', True)

In [3509]:
# Paired T test HPT
    # Shapiro wilk 
import scipy.stats as stats

# Baseline measurements
baseline = MR_no_exo['Net MR [W/kg]'].values

# Follow-up measurements
follow_up = MR_HPT['Net MR [W/kg]'].values

# Perform Shapiro-Wilk test on baseline measurements
_, baseline_p_value = stats.shapiro(baseline)
print("Baseline Shapiro-Wilk p-value:", baseline_p_value)

# Perform Shapiro-Wilk test on follow-up measurements
_, follow_up_p_value = stats.shapiro(follow_up)
print("Follow-up Shapiro-Wilk p-value:", follow_up_p_value)
print('significance', follow_up_p_value>0.05, baseline_p_value>0.05)

    # paired t 
import scipy.stats as stats

# Perform paired t-test
t_statistic, p_value = stats.ttest_rel(baseline, follow_up)

# Print the results
print("Paired t-test:")
print("p-value:", p_value)
'significance', p_value<0.05

Baseline Shapiro-Wilk p-value: 0.9997049570083618
Follow-up Shapiro-Wilk p-value: 0.29229259490966797
significance True True
Paired t-test:
p-value: 0.4299142657061783


('significance', False)

**PCA**

In [3510]:
features = list(Baseline_df.columns)[5:]

X = Baseline_df[features]

# normalizing features into z scores
X_norm = (X - X.mean(axis=0))/X.std(axis=0)

In [3511]:
# fit factor analyzer with principal components and varimax rotation
fa = FactorAnalyzer(rotation="varimax", n_factors=4, method='principal')
fa.fit(X_norm)

# get the rotated factor pattern
loadings2 = pd.DataFrame(np.round(fa.loadings_, 3), index=X_norm.columns, columns=[f"Factor{i+1}" for i in range(4)])
rotated_factor_pattern = loadings2[abs(loadings2) >= 0.80].dropna(how='all')

# display the results
rotated_factor_pattern = rotated_factor_pattern.drop(['mean_ss_time (s)No_Exo', 'mean_GC_range_AP (mm)No_Exo',
 'mean_vel_AP (mm/s)No_Exo', 'mean_GC_rms_acc_AP (mm/s2)No_Exo', 'cov_step_width (%)No_Exo'], axis=0)
rotated_factor_pattern.columns = ['Rythm', 'Efficiency', 'Body Weight Support', 'Dynamic Stability']
display(rotated_factor_pattern)
len(rotated_factor_pattern)
# df.to_excel(file_path, index=False)


,Rythm,Efficiency,Body Weight Support,Dynamic Stability
mean_step_time (s)No_Exo,0.897,NaN,NaN,NaN
mean_stride_time (s)No_Exo,0.897,NaN,NaN,NaN
cadence (step/min)No_Exo,-0.881,NaN,NaN,NaN
mean_swing_time (s)No_Exo,0.827,NaN,NaN,NaN
mean_stance_time (s)No_Exo,0.837,NaN,NaN,NaN
mean_ds_time (s)No_Exo,NaN,NaN,-0.802,NaN
cov_step_time (%)No_Exo,NaN,0.941,NaN,NaN
cov_stride_time (%)No_Exo,NaN,0.978,NaN,NaN
cov_swing_time (%)No_Exo,NaN,0.905,NaN,NaN
cov_stance_time (%)No_Exo,NaN,0.908,NaN,NaN


23

In [3512]:
rotated_factor_pattern.loc[['mean_step_time (s)No_Exo', 'mean_stride_time (s)No_Exo',
       'cadence (step/min)No_Exo', 'mean_swing_time (s)No_Exo',
       'mean_stance_time (s)No_Exo', 'mean_ds_time (s)No_Exo',
       'cov_step_time (%)No_Exo', 'cov_stride_time (%)No_Exo',
       'cov_swing_time (%)No_Exo', 'cov_stance_time (%)No_Exo',
       'swing_time_perctg (%)No_Exo', 'stance_time_perctg (%)No_Exo',
       'ds_time_perctg (%)No_Exo', 'ss_time_perctg (%)No_Exo',
       'mean_step_width (mm)No_Exo', 'mean_step_length (mm)No_Exo',
       'mean_stride_length (mm)No_Exo', 'cov_step_length (%)No_Exo',
       'cov_stride_length (%)No_Exo', 'mean_GC_range_ML (mm)No_Exo',
       'mean_vel_ML (mm/s)No_Exo', 'mean_GC_rms_acc_ML (mm/s2)No_Exo',
       'linear_momentum_ML (kg.m/s)No_Exo']]
       # .to_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/PCA_DF.xlsx', index=False)

,Rythm,Efficiency,Body Weight Support,Dynamic Stability
mean_step_time (s)No_Exo,0.897,NaN,NaN,NaN
mean_stride_time (s)No_Exo,0.897,NaN,NaN,NaN
cadence (step/min)No_Exo,-0.881,NaN,NaN,NaN
mean_swing_time (s)No_Exo,0.827,NaN,NaN,NaN
mean_stance_time (s)No_Exo,0.837,NaN,NaN,NaN
mean_ds_time (s)No_Exo,NaN,NaN,-0.802,NaN
cov_step_time (%)No_Exo,NaN,0.941,NaN,NaN
cov_stride_time (%)No_Exo,NaN,0.978,NaN,NaN
cov_swing_time (%)No_Exo,NaN,0.905,NaN,NaN
cov_stance_time (%)No_Exo,NaN,0.908,NaN,NaN


In [3513]:
# Calculate explained variance of each factor
explained_variance = fa.get_factor_variance()
explained_variance_df = pd.DataFrame(
    {
        'Factor': list(rotated_factor_pattern.columns),
        # 'Explained Variance': explained_variance[0],
        'Explained Variance %': np.round(explained_variance[1]*100, 2),
        'Cumulative Variance': np.round(explained_variance[2]*100, 2)
    }
)
display(explained_variance_df)

,Factor,Explained Variance %,Cumulative Variance
0,Rythm,25.83,25.83
1,Efficiency,17.93,43.76
2,Body Weight Support,15.04,58.80
3,Dynamic Stability,19.97,78.77


anthropo

In [3514]:
resp[['ID', 'Age', 'Height (m)', 'Weight (kg)', 'BMI (kg/m2)']]

,ID,Age,Height (m),Weight (kg),BMI (kg/m2)
0,1,21,1.77,77.0,24.577867
1,2,22,1.75,68.0,22.204082
2,3,21,1.92,70.0,18.988715
3,4,24,1.72,63.0,21.295295
7,9,28,1.89,83.0,23.235632
9,11,25,1.81,77.0,23.503556
10,12,59,1.77,77.0,24.577867
12,14,27,1.77,71.5,22.822305
14,16,28,1.67,59.5,21.334576
17,21,44,1.80,72.5,22.376543


In [3515]:
len(resp), len(non_resp)

(11, 8)

In [3516]:
# import pandas as pd
from scipy import stats

# Create sample dataframes
df1 = non_resp[['Age', 'Height (m)', 'Weight (kg)', 'BMI (kg/m2)']]

df2 = resp[['Age', 'Height (m)', 'Weight (kg)', 'BMI (kg/m2)']]

# Create an empty dataframe to store the results
final_df = pd.DataFrame(columns=['Variable', 'non-resp', 'resp', 'p-value'])

# Iterate over each column
for column in df1.columns:
    # Calculate mean and standard deviation for resp
    mean_group1 = df1[column].median()
    std_group1 = df1[column].quantile(0.75) - df1[column].quantile(0.25)

    # Calculate mean and standard deviation for non-resp
    mean_group2 = df2[column].median()
    std_group2 = df2[column].quantile(0.75) - df2[column].quantile(0.25)

    # Perform Mann-Whitney U test and get p-value
    p_value = stats.mannwhitneyu(df1[column], df2[column])[1]

    # Append the results to the final dataframe
    final_df = final_df.append({'Variable': column,
                                'non-resp': f"{round(mean_group1, 2)} ({round(std_group1, 2)})",
                                'resp': f"{round(mean_group2, 2)} ({round(std_group2, 2)})",
                                'p-value': round(p_value, 2)}, ignore_index=True)

final_df
# .to_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/AnthropoRespdf.xlsx', index=False)

,Variable,non-resp,resp,p-value
0,Age,23.5 (6.0),26.0 (5.0),0.40
1,Height (m),1.84 (0.07),1.77 (0.07),0.10
2,Weight (kg),77.5 (6.38),71.5 (11.5),0.09
3,BMI (kg/m2),22.53 (2.71),22.38 (2.05),0.54


In [3517]:
# import pandas as pd
from scipy import stats

# Create sample dataframes
df1 = SRC[['Age', 'Height (m)', 'Weight (kg)', 'BMI (kg/m2)']]


# Create an empty dataframe to store the results
final_df = pd.DataFrame(columns=['Variable', 'Value'])

# Iterate over each column
for column in df1.columns:
    # Calculate mean and standard deviation for resp
    mean_group1 = df1[column].mean()
    std_group1 = df1[column].std()

    # Append the results to the final dataframe
    final_df = final_df.append({'Variable': column,
                                'Value': f"{round(mean_group1, 2)} ({round(std_group1, 2)})"}, ignore_index=True)

final_df


,Variable,Value
0,Age,27.89 (9.72)
1,Height (m),1.8 (0.08)
2,Weight (kg),73.39 (7.97)
3,BMI (kg/m2),22.59 (1.76)


**SRC vs HPT**

In [3518]:
list(rotated_factor_pattern.index)

['mean_step_time (s)No_Exo',
 'mean_stride_time (s)No_Exo',
 'cadence (step/min)No_Exo',
 'mean_swing_time (s)No_Exo',
 'mean_stance_time (s)No_Exo',
 'mean_ds_time (s)No_Exo',
 'cov_step_time (%)No_Exo',
 'cov_stride_time (%)No_Exo',
 'cov_swing_time (%)No_Exo',
 'cov_stance_time (%)No_Exo',
 'swing_time_perctg (%)No_Exo',
 'stance_time_perctg (%)No_Exo',
 'ds_time_perctg (%)No_Exo',
 'ss_time_perctg (%)No_Exo',
 'mean_step_width (mm)No_Exo',
 'mean_step_length (mm)No_Exo',
 'mean_stride_length (mm)No_Exo',
 'cov_step_length (%)No_Exo',
 'cov_stride_length (%)No_Exo',
 'mean_GC_range_ML (mm)No_Exo',
 'mean_vel_ML (mm/s)No_Exo',
 'mean_GC_rms_acc_ML (mm/s2)No_Exo',
 'linear_momentum_ML (kg.m/s)No_Exo']

In [3519]:
saved_columns = ['ID',
 'Age',
 'Height (m)',
 'Weight (kg)',
 'BMI (kg/m2)',
 'mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_ds_time (s)',
 'mean_ss_time (s)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_width (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_GC_range_ML (mm)',
 'mean_GC_range_AP (mm)',
 'cov_range_ML (%)',
 'cov_range_AP (%)',
 'mean_vel_ML (mm/s)',
 'mean_vel_AP (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'mean_GC_rms_acc_AP (mm/s2)',
 'cov_rms_acc_ML (%)',
 'cov_rms_acc_AP (%)',
 'linear_momentum_ML (kg.m/s)',
 'linear_momentum_AP (kg.m/s)',
 'est_Vert_vel_redir (rad)',
 'est_mean_mech_work_pushoff (J/kg)',
 'est_inst_power_pushoff (W)',
 'est_cov_mech_work_pushoff (%)',
 'est_cov_inst_power_pushoff (%)']
columns_of_interest = ['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_ds_time (s)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']

In [3520]:
list1 = columns_of_interest#["apple", "banana", "orange", "grape"]
list2 = saved_columns#["banana", "grape", "kiwi", "pear"]

# Convert lists to sets
set1 = set(list1)
set2 = set(list2)

# Find the strings that are not present in both lists
not_in_list1 = set2 - set1
not_in_list2 = set1 - set2

# Convert the sets back to lists
result = list(not_in_list1.union(not_in_list2))

print(result)


['mean_ss_time (s)', 'mean_vel_AP (mm/s)', 'est_cov_mech_work_pushoff (%)', 'cov_step_width (%)', 'Weight (kg)', 'Height (m)', 'cov_range_AP (%)', 'ID', 'cov_rms_acc_ML (%)', 'est_inst_power_pushoff (W)', 'mean_GC_range_AP (mm)', 'linear_momentum_AP (kg.m/s)', 'est_Vert_vel_redir (rad)', 'mean_GC_rms_acc_AP (mm/s2)', 'est_mean_mech_work_pushoff (J/kg)', 'Age', 'cov_rms_acc_AP (%)', 'BMI (kg/m2)', 'cov_range_ML (%)', 'est_cov_inst_power_pushoff (%)']


In [3521]:
# SRCstat = SRC #
# SRCstat.columns = saved_columns
# HPTstat = HPT
# HPTstat.columns = saved_columns
# no_exostat = no_exo
# no_exostat.columns = saved_columns
# no_exostat = no_exostat[columns_of_interest]
# SRCstat = SRCstat[columns_of_interest]
# HPTstat = HPTstat[columns_of_interest]
# # import pandas as pd
# from scipy import stats
# # Create sample dataframes
# df1 = SRCstat

# df2 = HPTstat

# # Create an empty dataframe to store the results
# final_df = pd.DataFrame(columns=['Variable', 'SRC', 'HPT', 'p-value'])

# # Iterate over each column
# for column in df1.columns:
#     # Calculate mean and standard deviation for resp
#     mean_group1 = df1[column].mean()
#     std_group1 = df1[column].std()

#     # Calculate mean and standard deviation for non-resp
#     mean_group2 = df2[column].mean()
#     std_group2 = df2[column].std()

#     # Perform Mann-Whitney U test and get p-value
#     p_value = round(stats.ttest_rel(df1[column], df2[column])[1], 3)

#     # Append the results to the final dataframe
#     final_df = final_df.append({'Variable': column,
#                                 'SRC': f"{mean_group1} ({std_group1})",
#                                 'HPT': f"{mean_group2} ({std_group2})",
#                                 'p-value': p_value}, ignore_index=True)

# final_df.loc[final_df['p-value'] <0.06]

In [3522]:
import pandas as pd
from scipy.stats import friedmanchisquare
SRCstat = SRC #
SRCstat.columns = saved_columns
HPTstat = HPT
HPTstat.columns = saved_columns
no_exostat = no_exo
no_exostat.columns = saved_columns
no_exostat = no_exostat[columns_of_interest]
SRCstat = SRCstat[columns_of_interest]
HPTstat = HPTstat[columns_of_interest]
# Create sample dataframes
df1 = SRCstat
df2 = HPTstat
df3 = no_exostat

# Create an empty dataframe to store the results
final_df = pd.DataFrame(columns=['Variable', 'no_exo', 'SRC', 'HPT', 'p-value'])

# Iterate over each column
for column in df1.columns:
    # Calculate mean and standard deviation for each dataframe
    mean_group1 = df1[column].median()
    std_group1 = df1[column].quantile(0.75) - df1[column].quantile(0.25)

    mean_group2 = df2[column].median()
    std_group2 = df2[column].quantile(0.75) - df2[column].quantile(0.25)

    mean_group3 = df3[column].median()
    std_group3 = df3[column].quantile(0.75) - df3[column].quantile(0.25)

    # Perform the Friedman test and get p-value
    p_value = round(friedmanchisquare(df1[column], df2[column], df3[column]).pvalue, 2)

    # Append the results to the final dataframe
    final_df = final_df.append({'Variable': column,
                                'no_exo': f"{mean_group3:.2f} ({std_group3:.2f})",
                                'SRC': f"{mean_group1:.2f} ({std_group1:.2f})",
                                'HPT': f"{mean_group2:.2f} ({std_group2:.2f})",
                                'p-value': p_value}, ignore_index=True)

ordered = ['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']
final_df = final_df.sort_values(by='Variable', key=lambda x: x.map(dict(zip(ordered, range(len(ordered))))))

final_df['Variable'] = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']
final_df
# .loc[final_df['p-value'] <0.06]

,Variable,no_exo,SRC,HPT,p-value
0,Step Time (s),0.59 (0.03),0.60 (0.04),0.59 (0.03),0.00
1,Stride Time (s),1.18 (0.05),1.20 (0.08),1.19 (0.06),0.00
2,Step Frequency (step/min),101.00 (4.00),100.00 (6.50),101.00 (5.00),0.00
3,Swing Time (s),0.42 (0.03),0.43 (0.03),0.42 (0.03),0.00
4,Stance Time (s),0.76 (0.02),0.77 (0.05),0.76 (0.04),0.00
15,Step Length (mm),362.47 (28.75),371.19 (23.40),365.79 (27.90),0.00
16,Stride Length (mm),724.92 (57.52),742.39 (46.82),731.56 (55.66),0.00
6,Step Time Variability (%),3.03 (0.88),3.04 (0.90),3.03 (0.69),0.33
7,Stride Time Variability (%),1.84 (0.45),1.89 (0.68),1.89 (0.52),0.23
8,Swing Time Variability (%),4.45 (1.04),4.65 (0.75),4.53 (0.49),0.69


**resp vs non resp**

In [3523]:
# import pandas as pd
from scipy import stats
non_resp_stat = non_resp
non_resp_stat_no_exo = non_resp_noexo
non_resp_stat.columns = saved_columns
non_resp_stat_no_exo.columns = saved_columns
non_resp_stat = non_resp_stat[columns_of_interest]
non_resp_stat_no_exo = non_resp_stat_no_exo[columns_of_interest]

resp_stat = resp
resp_stat.columns = saved_columns
resp_stat = resp_stat[columns_of_interest]
resp_stat_no_exo = resp_noexo
resp_stat_no_exo.columns = saved_columns
resp_stat_no_exo = resp_stat_no_exo[columns_of_interest]

change_resp, change_non_resp = resp_stat_no_exo - resp_stat, non_resp_stat_no_exo - non_resp_stat
# Create sample dataframes
df1 = change_non_resp

df2 = change_resp

# Create an empty dataframe to store the results
final_df = pd.DataFrame(columns=['Variable', 'non-resp change', 'resp change', 'p-value'])

# Iterate over each column
for column in df1.columns:
    # Calculate mean and standard deviation for resp
    mean_group1 = df1[column].median()
    std_group1 = df1[column].quantile(0.75) - df1[column].quantile(0.25)

    # Calculate mean and standard deviation for non-resp
    mean_group2 = df2[column].median()
    std_group2 = df2[column].quantile(0.75) - df2[column].quantile(0.25)

    # Perform Mann-Whitney U test and get p-value
    p_value = round(stats.mannwhitneyu(df1[column], df2[column], alternative= 'greater')[1], 2) #‘two-sided’, ‘less’, ‘greater’

    # Append the results to the final dataframe
    final_df = final_df.append({'Variable': column,
                                'non-resp change': f"{round(mean_group1, 2)} ({round(std_group1, 2)})",
                                'resp change': f"{round(mean_group2, 2)} ({round(std_group2, 2)})",
                                'p-value': p_value}, ignore_index=True)

final_df
# .loc[final_df['p-value'] <0.06]

ordered = ['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']
final_df = final_df.sort_values(by='Variable', key=lambda x: x.map(dict(zip(ordered, range(len(ordered))))))

final_df['Variable'] = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']
final_df['Variable'] = ['Δ'+i for i in list(final_df['Variable'])]
final_df
# .loc[final_df['p-value'] <0.06]

,Variable,non-resp change,resp change,p-value
0,ΔStep Time (s),0.0 (0.02),-0.01 (0.01),0.05
1,ΔStride Time (s),0.0 (0.03),-0.03 (0.02),0.05
2,ΔStep Frequency (step/min),0.0 (2.75),2.0 (2.0),0.97
3,ΔSwing Time (s),-0.0 (0.01),-0.01 (0.01),0.12
4,ΔStance Time (s),0.0 (0.02),-0.02 (0.01),0.05
15,ΔStep Length (mm),1.14 (9.6),-8.57 (6.13),0.05
16,ΔStride Length (mm),2.3 (19.28),-17.14 (12.24),0.05
6,ΔStep Time Variability (%),0.06 (0.83),0.06 (0.7),0.45
7,ΔStride Time Variability (%),-0.24 (1.14),-0.15 (0.47),0.73
8,ΔSwing Time Variability (%),-0.36 (0.83),-0.19 (1.22),0.48


**Correl Domains with mr reduction (controlling for respondent)**

In [3524]:
change = no_exostat - SRCstat
# Create sample dataframes
df1 = change

df2 = change_non_resp[['mean_step_time (s)']]

# Create an empty dataframe to store the results
final_df = pd.DataFrame(columns=['Variable', 'param change correl with metabolic cost red', 'p-value',])

# Iterate over each column
for column in df1.columns:
    # # Calculate mean and standard deviation for resp
    mean_group1 = df1[column].median()
    std_group1 = df1[column].quantile(0.75) - df1[column].quantile(0.25)

    # # Calculate mean and standard deviation for non-resp
    # mean_group2 = df2[column].mean()
    # std_group2 = df2[column].std()

    # Perform Mann-Whitney U test and get p-value
    r1 = spearmanr(df1[column].values, MR_change.red.values)[0]
    r2 = spearmanr(df1[column].values, MR_change.red.values)[1]
    
    # Append the results to the final dataframe
    final_df = final_df.append({'Variable': column,
                                # 'value': f"{round(mean_group1, 2)} ({round(std_group1, 2)})",
                                'R': round(r1, 2),
                                'p-value': round(r2, 2)}, ignore_index=True)

final_df
# .loc[final_df['p-value'] <0.06]

ordered = ['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']
final_df = final_df.sort_values(by='Variable', key=lambda x: x.map(dict(zip(ordered, range(len(ordered))))))

final_df['Variable'] = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']
final_df['Variable'] = ['Δ'+i for i in list(final_df['Variable'])]
final_df
# .to_excel('/Users/cyrilleetude/Downloads/e_walk_data_analysis/Data_analysis/R.xlsx', index=False)
# .loc[final_df['p-value'] <0.06]

**Viz**

In [3525]:
# Normalization for radar and boxplots



#SRCvsHPT

no_exostat = no_exostat[['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']]
no_exostat.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']



# normalizing features into z scores
no_exostat_norm = (no_exostat - no_exostat.mean(axis=0))/no_exostat.std(axis=0)

SRCstat  = SRCstat[['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']]
SRCstat.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']



# normalizing features into z scores
SRCstat_norm = (SRCstat - no_exostat.mean(axis=0))/no_exostat.std(axis=0)


HPTstat  = HPTstat[['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']]
HPTstat.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']



# normalizing features into z scores
HPTstat_norm = (HPTstat - no_exostat.mean(axis=0))/no_exostat.std(axis=0)

#get domain df
rythm_no_exo = no_exostat_norm.iloc[:, 0:7]
rythm_SRC = SRCstat_norm.iloc[:, 0:7]
rythm_HPT = HPTstat_norm.iloc[:, 0:7]

efficiency_no_exo = no_exostat_norm.iloc[:, 7:13]
efficiency_SRC = SRCstat_norm.iloc[:, 7:13]
efficiency_HPT = HPTstat_norm.iloc[:, 7:13]

body_weight_support_no_exo = no_exostat_norm.iloc[:, 13:18]
body_weight_support_SRC = SRCstat_norm.iloc[:, 13:18]
body_weight_support_HPT = HPTstat_norm.iloc[:, 13:18]

dynamic_stability_no_exo = no_exostat_norm.iloc[:, 18:23]
dynamic_stability_SRC = SRCstat_norm.iloc[:, 18:23]
dynamic_stability_HPT = HPTstat_norm.iloc[:, 18:23]

#Get arrays for box plot (then will be used for radar by concatenaining by condition)
rythm_no_exo_array = np.array(rythm_no_exo.median())
bws_no_exo_array = np.array(body_weight_support_no_exo.median())
efficiency_no_exo_array = np.array(efficiency_no_exo.median())
stability_no_exo_array = np.array(dynamic_stability_no_exo.median())


rythm_SRC_array = np.array(rythm_SRC.median())
bws_SRC_array = np.array(body_weight_support_SRC.median())
efficiency_SRC_array = np.array(efficiency_SRC.median())
stability_SRC_array = np.array(dynamic_stability_SRC.median())


rythm_HPT_array = np.array(rythm_HPT.median())
bws_HPT_array = np.array(body_weight_support_HPT.median())
efficiency_HPT_array = np.array(efficiency_HPT.median())
stability_HPT_array = np.array(dynamic_stability_HPT.median())

import plotly.graph_objects as go
import numpy as np

# Generate or use your existing arrays
array1 = np.random.randn(100)
array2 = np.random.randn(100)
array3 = np.random.randn(100)
array4 = np.random.randn(100)
array5 = np.random.randn(100)
array6 = np.random.randn(100)

data = [-rythm_no_exo_array, -rythm_HPT_array, -rythm_SRC_array, efficiency_no_exo_array, efficiency_HPT_array, efficiency_SRC_array,
bws_no_exo_array, bws_HPT_array, bws_SRC_array, -stability_no_exo_array, -stability_HPT_array, -stability_SRC_array]
labels = ['Box 1', 'Box 2', 'Box 3', 'Box 4', 'Box 5', 'Box 6', 'Box 7', 'Box 8', 'Box 9', 'Box 10', 'Box 11', 'Box 12']

# Define the colors for the boxes
colors = ['red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue', 'red', 'green', 'blue']

traces = []
for i in range(len(data)):
    trace = go.Box(
        y=data[i],
        name=labels[i],
        showlegend=False, 
        fillcolor='rgba(0, 0, 0, 0)',  # Make the box transparent
        line=dict(color=colors[i], width=1),  # Set the border color and width
    
        marker=dict(size=0),  # Set marker size to 0
        boxpoints=False,  # Disable individual data points
        jitter=0  )# Set jitter to 0
    traces.append(trace)

# Calculate medians of the first three boxes
medians = [np.median(i) for i in data]
median1 = np.median(array1)
median2 = np.median(array2)
median3 = np.median(array3)

# Calculate medians of the last three boxes
median4 = np.median(array4)
median5 = np.median(array5)
median6 = np.median(array6)

# Create dashed black lines connecting the medians
traces.append(go.Scatter(x=labels[:3], y=medians[:3],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[3:6], y=medians[3:6],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[6:9], y=medians[6:9],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[9:12], y=medians[9:12],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

layout = go.Layout(
    title='Box Plot SRC vs HPT',
    yaxis=dict(title='Z-Scores', showticklabels=True),
    xaxis=dict(showticklabels=False)
)

fig = go.Figure(data=traces, layout=layout)

fig.update_layout(template = 'plotly_white', height=800, width=1400, showlegend=False)
# fig.update_yaxes(range = [-10*10**(-15),15*10**(-15)])

fig.show()



#Radar plot
no_exo_array = np.concatenate((rythm_no_exo_array, efficiency_no_exo_array, bws_no_exo_array, stability_no_exo_array))
HPT_array = np.concatenate((rythm_HPT_array, efficiency_HPT_array, bws_HPT_array, stability_HPT_array))
SRC_array = np.concatenate((rythm_SRC_array, efficiency_SRC_array, bws_SRC_array, stability_SRC_array))
# Define the columns for labeling
categories = ['Step Time (s)', 'Stride Time (s)', 'Step Frequency (step/min)', 'Swing Time (s)', 'Stance Time (s)',
           'Step Length (mm)', 'Stride Length (mm)', 'Step Time Variability (%)', 'Stride Time Variability (%)',
           'Swing Time Variability (%)', 'Stance Time Variability (%)', 'Step Length Variability (%)',
           'Stride Length Variability (%)', 'Double Stance Time (s)', 'Swing Time Percentage (GC %)',
           'Stance Time Percentage (GC %)', 'Double Stance Time Percentage (GC %)',
           'Single Stance Time Percentage (GC %)', 'Step Width (mm)', 'COM ML GC Range (mm)', 'COM ML Velocity (mm/s)',
           'COM ML GC RMS Acc (mm/s2)', 'COM ML Linear Momentum (kg.m/s)']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=no_exo_array,
      theta=categories,
      mode='markers',  # Add markers for each data point
      marker=dict(color='red', size=5, opacity = 0.5),  # Set the marker color and size
      fill='toself',
      name='No Exo',
      fillcolor='rgba(255, 0, 0, 0.05)',
      line=dict(color='red', dash='dash', width = 0)
))
fig.add_trace(go.Scatterpolar(
      r=HPT_array,
      theta=categories,
      mode='markers',  # Add markers for each data point
      marker=dict(color='green', size=5, opacity = 0.5),  # Set the marker color and size
      fill='toself',
      name='HPT',
      fillcolor='rgba(0, 128, 0, 0.15)',
      line=dict(color='green', width = 0)
))
fig.add_trace(go.Scatterpolar(
      r=SRC_array,
      theta=categories,
      mode='markers',  # Add markers for each data point
      marker=dict(color='blue', size=5, opacity = 0.5),  # Set the marker color and size
      fill='toself',
      name='SRC',
      fillcolor='rgba(0, 0, 255, 0.15)',
      line=dict(color='blue', width = 0)
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-0.5, 1.5]
    )),
  showlegend=False
)
fig.update_layout(template = 'plotly_white', height=1300, width=1600, showlegend=False)
fig.show()



In [3526]:

# # Calculate medians of the first three boxes
# medians = [np.median(i) for i in data]
# median1 = np.median(array1)
# median2 = np.median(array2)
# median3 = np.median(array3)

# # Calculate medians of the last three boxes
# median4 = np.median(array4)
# median5 = np.median(array5)
# median6 = np.median(array6)

# # Create dashed black lines connecting the medians
# traces.append(go.Scatter(x=labels[:3], y=medians[:3],
#                          mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

# traces.append(go.Scatter(x=labels[3:6], y=medians[3:6],
#                          mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

# traces.append(go.Scatter(x=labels[6:9], y=medians[6:9],
#                          mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

# traces.append(go.Scatter(x=labels[9:12], y=medians[9:12],
#                          mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))


In [3527]:
data = [MR_no_exo['Net MR [W/kg]'].values, MR_HPT['Net MR [W/kg]'].values, MR_SRC['Net MR [W/kg]'].values]
labels = ['Box 1', 'Box 2', 'Box 3']

# Define the colors for the boxes
colors = ['red', 'green', 'blue']

traces = []
for i in range(len(data)):
    trace = go.Box(
        y=data[i],
        name=labels[i],
        showlegend=False, 
        fillcolor='rgba(0, 0, 0, 0)',  # Make the box transparent
        line=dict(color=colors[i], width=1),  # Set the border color and width
    
        marker=dict(size=0),  # Set marker size to 0
        boxpoints=False,  # Disable individual data points
        jitter=0  )# Set jitter to 0
    traces.append(trace)


    # # Add dots for each box
    # dot_trace = go.Scatter(
    #     x=[i + 1] * len(data[i]),
    #     y=data[i],
    #     mode='markers',
    #     name=labels[i],
    #     marker=dict(color=colors[i], size=8, opacity = 0.4)
    # )
    # traces.append(dot_trace)

# Calculate medians of the first three boxes
medians = [np.median(i) for i in data]

# Create dashed black lines connecting the medians
traces.append(go.Scatter(x=labels[:3], y=medians[:3],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))
layout = go.Layout(
    title='Box Plot SRC vs HPT MR',
    yaxis=dict(title='Net MR [W/kg]', showticklabels=True),
    xaxis=dict(showticklabels=False)
)

fig = go.Figure(data=traces, layout=layout)

fig.update_layout(template = 'plotly_white', height=800, width=1000, showlegend=False)
fig.update_yaxes(range = [1,4])

fig.show()

In [3528]:
# Normalization for radar and boxplots



#no_resp vs resp

change_resp  = change_resp[['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']]
change_resp.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']



# normalizing features into z scores
change_resp_norm = (change_resp - change_resp.mean(axis=0))/change_resp.std(axis=0)

change_non_resp  = change_non_resp[['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)']]
change_non_resp.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']



# normalizing features into z scores
change_non_resp_norm = (change_non_resp - change_resp.mean(axis=0))/change_resp.std(axis=0)




#get domain df
rythm_change_resp = change_resp_norm.iloc[:, 0:7]
efficiency_change_resp = change_resp_norm.iloc[:, 7:13]
body_weight_support_change_resp = change_resp_norm.iloc[:, 13:18]
dynamic_stability_change_resp = change_resp_norm.iloc[:, 18:23]

#Get arrays for box plot (then will be used for radar by concatenaining by condition)
rythm_change_resp_array = np.array(rythm_change_resp.median())
bws_change_resp_array = np.array(body_weight_support_change_resp.median())
efficiency_change_resp_array = np.array(efficiency_change_resp.median())
stability_change_resp_array = np.array(dynamic_stability_change_resp.median())

#get domain df
rythm_change_non_resp = change_non_resp_norm.iloc[:, 0:7]
efficiency_change_non_resp = change_non_resp_norm.iloc[:, 7:13]
body_weight_support_change_non_resp = change_non_resp_norm.iloc[:, 13:18]
dynamic_stability_change_non_resp = change_non_resp_norm.iloc[:, 18:23]

#Get arrays for box plot (then will be used for radar by concatenaining by condition)
rythm_change_non_resp_array = np.array(rythm_change_non_resp.median())
bws_change_non_resp_array = np.array(body_weight_support_change_non_resp.median())
efficiency_change_non_resp_array = np.array(efficiency_change_non_resp.median())
stability_change_non_resp_array = np.array(dynamic_stability_change_non_resp.median())



import plotly.graph_objects as go
import numpy as np

data = [rythm_change_resp_array, rythm_change_non_resp_array, efficiency_change_resp_array, efficiency_change_non_resp_array,
bws_change_resp_array, bws_change_non_resp_array, stability_change_resp_array, stability_change_non_resp_array]
data = [-i for i in data]
labels = ['Box 1', 'Box 2', 'Box 3', 'Box 4', 'Box 5', 'Box 6', 'Box 7', 'Box 8']

# Define the colors for the boxes
colors = ['rgba(0, 0, 0, 0.5)', 'orange', 'rgba(0, 0, 0, 0.5)', 'orange', 'rgba(0, 0, 0, 0.5)', 'orange', 'rgba(0, 0, 0, 0.5)', 'orange']

traces = []
for i in range(len(data)):
    trace = go.Box(
        y=data[i],
        name=labels[i],
        showlegend=False, 
        fillcolor='rgba(0, 0, 0, 0)',  # Make the box transparent
        line=dict(color=colors[i], width=1),  # Set the border color and width
    
        marker=dict(size=0),  # Set marker size to 0
        boxpoints=False,  # Disable individual data points
        jitter=0  )# Set jitter to 0
    traces.append(trace)

# Calculate medians of the first three boxes
medians = [np.median(i) for i in data]

# Calculate medians of the last three boxes

# Create dashed black lines connecting the medians
traces.append(go.Scatter(x=labels[:2], y=medians[:2],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[2:4], y=medians[2:4],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[4:6], y=medians[4:6],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

traces.append(go.Scatter(x=labels[6:8], y=medians[6:8],
                         mode='lines', showlegend=False,  line=dict(color='black', dash='dash', width = 1.5)))

layout = go.Layout(
    title='Box Plot Resp vs non Resp',
    yaxis=dict(title='Z-Scores', showticklabels=True),
    xaxis=dict(showticklabels=False)
)

fig = go.Figure(data=traces, layout=layout)

fig.update_layout(template = 'plotly_white', height=600, width=1000, showlegend=False)
# fig.update_yaxes(range = [-10*10**(-15),15*10**(-15)])

fig.show()



# #Radar plot
# change_resp_array = np.concatenate((rythm_change_resp_array, efficiency_change_resp_array, bws_change_resp_array, stability_change_resp_array))
# change_non_resp_array = np.concatenate((rythm_change_non_resp_array, efficiency_change_non_resp_array, bws_change_non_resp_array, stability_change_non_resp_array))
# # Define the columns for labeling
# categories = ['ΔStep Time (s)', 'ΔStride Time (s)', 'ΔStep Frequency (step/min)',
#        'ΔSwing Time (s)', 'ΔStance Time (s)', 'ΔStep Length (mm)',
#        'ΔStride Length (mm)', 'ΔStep Time Variability (%)',
#        'ΔStride Time Variability (%)', 'ΔSwing Time Variability (%)',
#        'ΔStance Time Variability (%)', 'ΔStep Length Variability (%)',
#        'ΔStride Length Variability (%)', 'ΔDouble Stance Time (s)',
#        'ΔSwing Time Percentage (GC %)', 'ΔStance Time Percentage (GC %)',
#        'ΔDouble Stance Time Percentage (GC %)',
#        'ΔSingle Stance Time Percentage (GC %)', 'ΔStep Width (mm)',
#        'ΔCOM ML GC Range (mm)', 'ΔCOM ML Velocity (mm/s)',
#        'ΔCOM ML GC RMS Acc (mm/s2)', 'ΔCOM ML Linear Momentum (kg.m/s)']

# fig = go.Figure()

# fig.add_trace(go.Scatterpolar(
#       r=change_resp_array,
#       theta=categories,
#       mode='markers',  # Add markers for each data point
#       marker=dict(color='black', size=5, opacity = 0.5),  # Set the marker color and size
#       fill='toself',
#       name='resp',
#       fillcolor='rgba(0, 0, 0, 0.15)',
#       line=dict(color='black', dash='dash', width = 0)
# ))
# fig.add_trace(go.Scatterpolar(
#       r=change_non_resp_array,
#       theta=categories,
#       mode='markers',  # Add markers for each data point
#       marker=dict(color='orange', size=5, opacity = 0.5),  # Set the marker color and size
#       fill='toself',
#       name='non-resp',
#       fillcolor='rgba(255, 165, 0, 0.5)',
#       line=dict(color='orange', width = 0)
# ))

# fig.update_layout(
#   polar=dict(
#     radialaxis=dict(
#       visible=True,
#       range=[-0.5, 2.2]
#     )),
#   showlegend=False
# )
# fig.update_layout(template = 'plotly_white', height=1300, width=1600, showlegend=False)
# fig.show()



In [3529]:
change_norm  = change.reindex(columns =['mean_step_time (s)',
 'mean_stride_time (s)',
 'cadence (step/min)',
 'mean_swing_time (s)',
 'mean_stance_time (s)',
 'mean_step_length (mm)',
 'mean_stride_length (mm)',
 'cov_step_time (%)',
 'cov_stride_time (%)',
 'cov_swing_time (%)',
 'cov_stance_time (%)',
 'cov_step_length (%)',
 'cov_stride_length (%)',
 'mean_ds_time (s)',
 'swing_time_perctg (%)',
 'stance_time_perctg (%)',
 'ds_time_perctg (%)',
 'ss_time_perctg (%)',
 'mean_step_width (mm)',
 'mean_GC_range_ML (mm)',
 'mean_vel_ML (mm/s)',
 'mean_GC_rms_acc_ML (mm/s2)',
 'linear_momentum_ML (kg.m/s)'])
change_norm.columns = ['Step Time (s)',
 'Stride Time (s)',
 'Step Frequency (step/min)',
 'Swing Time (s)',
 'Stance Time (s)',
 'Step Length (mm)',
 'Stride Length (mm)',
 'Step Time Variability (%)',
 'Stride Time Variability (%)',
 'Swing Time Variability (%)',
 'Stance Time Variability (%)',
 'Step Length Variability (%)',
 'Stride Length Variability (%)',
 'Double Stance Time (s)',
 'Swing Time Percentage (GC %)',
 'Stance Time Percentage (GC %)',
 'Double Stance Time Percentage (GC %)',
 'Single Stance Time Percentage (GC %)',
 'Step Width (mm)',
 'COM ML GC Range (mm)',
 'COM ML Velocity (mm/s)',
 'COM ML GC RMS Acc (mm/s2)',
 'COM ML Linear Momentum (kg.m/s)']


In [3530]:
# HPT controller

import plotly.graph_objects as go
import numpy as np

Torque = np.array([-2, -4, -4, 0, 0, 4, 4, 0, 0, -2])
GC = np.array([0, 10, 20, 20, 40, 60, 80, 80, 90, 100])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=GC,
    y=Torque,
    mode='lines',
    line=dict(color='blue', width=4),
))

for template in ["plotly_white"]:
    fig.update_layout(template=template, title_text='')

fig.update_layout(showlegend=False)

# Hide y-axis graduations except 0
fig.update_yaxes(tickvals=[0])

# Display x-axis with intervals of 10
fig.update_xaxes(dtick=20)

fig.update_yaxes(title_text="")
fig.update_xaxes(title_text="GC%")
fig.update_layout(width=800, height=400)
# Show the plot
fig.show()


In [3531]:
# SRC controller

import plotly.graph_objects as go
import numpy as np

Torque = np.array([4, -4, -4, 4, 4])
GC = np.array([0, 15, 50, 60, 100])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=GC,
    y=Torque,
    mode='lines',
    line=dict(color='blue', width=4),
))

for template in ["plotly_white"]:
    fig.update_layout(template=template, title_text='')

fig.update_layout(showlegend=False)

# Hide y-axis graduations except 0
fig.update_yaxes(tickvals=[0])

# Display x-axis with intervals of 10
fig.update_xaxes(dtick=20)

fig.update_yaxes(title_text="")
fig.update_xaxes(title_text="GC%")
fig.update_layout(width=800, height=400)
# Show the plot
fig.show()
